In [107]:
# import dependencies
import pandas as pd
import numpy as np
import sys
from datetime import datetime
import getpass
from IPython.display import display


# # For MySQL
# from sqlalchemy import create_engine
# import pandas as pd
# import pymysql

# For PostgreSQL
from sqlalchemy import create_engine
import pandas as pd
import psycopg2

# # For SQL Server
# from sqlalchemy import create_engine
# import pandas as pd
# import pyodbc

In [91]:
import psycopg2

username = 'gmb'
password = getpass.getpass('Enter password:')
public_ip_address = '34.133.118.52'
db_name = 'gmb'

# Create a connection
conn = psycopg2.connect(
    dbname=db_name,
    user=username,
    password=password,
    host=public_ip_address
)

# Create a cursor
cur = conn.cursor()

Enter password:········


# Interactive Database Exploration Tool

This tool provides an interactive prompt to the users, allowing them to explore the database by viewing table names, column names, and a sample of the data points. Users can choose to view specific details based on their preferences.


### Show Table names

In [92]:
def show_tables (cursor):
    
    query_tables = """ 
    select tablename
    from pg_catalog.pg_tables
    where schemaname != 'pg_catalog' AND
          schemaname != 'information_schema';
    """
    cursor.execute(query_tables)
    tables = cursor.fetchall()
    if tables:
        for table in tables:
            print(table[0])
    else:
        print("No tables found.")

### Show Sample Data Points

In [108]:
def show_sample_data(cursor, table_name):
    
    query_sample = f"select * from {table_name} limit 5;"
    
    df_sample = pd.read_sql(query_sample, cursor.connection)
    df_sample = pd.DataFrame(df_sample)
    print(f"Sample data from {table_name}:")
    display(df_sample)


In [95]:
################### show tables
if input("Do you like to see the table names? (Yes/No): ").strip().lower() == "yes":
    show_tables(cur)

Do you like to see the table names? (Yes/No): yes
activity_category
activity_activitytemplateusage
activity_customprompt
activity_activitytemplate
activity_activitytemplate_categories
activity_templateevent
activity_note
activity_habit
activity_negativebehaviour
activity_plan
activity_templateevent_prompts
activity_event_prompts
auth_permission
auth_group_permissions
auth_group
activity_event
billing_subscriptionplan
avatar_avatareyes
avatar_avatarhair
avatar_avatarmouth
billing_paymentsubscription
challenge_challengeicon
challenge_challenge_categories
billing_subscriptionplan_segmentation
campaigns_campaign
campaigns_campaign_subscription_plans
billing_paymentcustomer
challenge_challengeinvite
challenge_challengechatmessage
avatar_avatarbody
challenge_challenge
chat_chatmessage
chat_thread
community_communityquestion
connected_app_app
django_admin_log
community_community
group_group_categories
challenge_challengeparticipant
django_session
django_migrations
django_site
django_content_t

In [109]:
#################### show data sample
if input("Would you like to access and see a sample of the data points (yes/no): ").strip().lower() == "yes":
    
    table_name = input("Please enter the table name: ")
    show_sample_data(cur, table_name)

Would you like to access and see a sample of the data points (yes/no): yes
Please enter the table name: scoring_level
Sample data from scoring_level:


/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,name,level,category,xp_requirement,color
0,9,Doughy,1,1,0,None
1,10,Average,2,1,100,None
2,11,Trainee,3,1,400,None
3,12,Four-Pack Buddy,4,1,1000,None
4,13,Fitness Assistant,5,1,2000,None


# Anonymization

In [5]:
query = """

SELECT distinct table_name
        FROM information_schema.columns
WHERE column_name = 'first_name' 
   OR column_name = 'last_name'
   OR column_name = 'phone'
   OR column_name = 'email'
   OR column_name = 'zipcode'
   OR column_name = 'password' 
"""
sensitive_tables = pd.read_sql(query, conn)
sensitive_tables

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,table_name
0,billing_paymentcustomer
1,challenge_challengeinvite
2,group_groupinvite
3,matching_matchingsettings
4,member_user
5,social_auth_code


### billing_paymentcustomer


In [6]:
query_billing = "select * from billing_paymentcustomer"
biling_paymentcustomer_df = pd.read_sql(query_billing, conn)

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [8]:
biling_paymentcustomer_anonydf = biling_paymentcustomer_df.copy()

biling_paymentcustomer_anonydf['first_name'] = biling_paymentcustomer_anonydf['id']\
.apply(lambda x: 'FirstUser{}'.format(x))

biling_paymentcustomer_anonydf['last_name'] = biling_paymentcustomer_anonydf['id']\
.apply(lambda x: 'LastUser{}'.format(x))

biling_paymentcustomer_anonydf['email'] = biling_paymentcustomer_anonydf['id']\
.apply(lambda x: 'user{}@example.com'.format(x))

if 'phone' in biling_paymentcustomer_anonydf.columns:
    biling_paymentcustomer_anonydf['phone'] = biling_paymentcustomer_anonydf['id']\
    .apply(lambda x: '123456{}'.format(x))
    
if 'created_from_ip' in biling_paymentcustomer_anonydf.columns:
    biling_paymentcustomer_anonydf['created_from_ip'] = "192.168.1.1" 
    


**About anonymizing IP address:**
"192.168.1.1" is a commonly used default IP address for many routers and home networking equipment. It falls within the range of IP addresses (192.168.0.0 to 192.168.255.255) reserved for private networks, meaning these addresses are not routable on the public internet and are often used for local network devices.

In [9]:
db_url = f"postgresql://{username}:{password}@{public_ip_address}/{db_name}"

# SQLAlchemy engine
engine = create_engine(db_url)

when using the `to_sql()` method in pandas, you need to provide an SQLAlchemy engine, not a psycopg2 connection. This is because pandas uses SQLAlchemy to handle SQL dialects.

In [10]:
biling_paymentcustomer_anonydf.to_sql('billing_paymentcustomer_anonymized', engine, if_exists='replace', index=False)


590

### challenge_challengeinvite

In [11]:
query_challenge = "select * from challenge_challengeinvite" 
challenge_challengeinvite_df = pd.read_sql(query_challenge, conn)
challenge_challengeinvite_df

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,email,status,created_at,updated_at,challenge_id,invited_by_id,user_id
0,12,None,20,2020-04-01 23:12:53.059068+00:00,2020-04-04 01:22:50.061822+00:00,158,2,9033
1,3,None,10,2020-03-14 23:41:45.580212+00:00,2020-03-14 23:41:45.580235+00:00,124,2,9065
2,5,None,10,2020-03-14 23:44:36.088580+00:00,2020-03-14 23:44:36.088598+00:00,117,2,17
3,2,None,20,2020-03-14 20:23:03.564005+00:00,2020-03-14 23:45:46.688291+00:00,117,2,9237
4,4,None,20,2020-03-14 23:44:35.956449+00:00,2020-03-15 00:07:48.216763+00:00,124,2,17
...,...,...,...,...,...,...,...,...
485,552,None,20,2023-03-12 08:12:37.341775+00:00,2023-03-12 08:46:28.135503+00:00,843,2,13382
486,555,None,10,2023-03-27 06:11:07.022965+00:00,2023-03-27 06:11:07.022989+00:00,838,2,13427
487,554,None,10,2023-03-27 06:11:07.022602+00:00,2023-03-27 06:11:07.022638+00:00,842,2,13427
488,556,None,20,2023-05-02 08:07:59.558115+00:00,2023-05-02 16:42:18.068892+00:00,848,2,12858


In [12]:
challenge_challengeinvite_df.email.unique()

array([None], dtype=object)

No need to anonymize the `challenge_challengeinvite` table currently since the `email` column consists of no emails.

### group_groupinvite

In [13]:
query_groupinvite = "select * from group_groupinvite" 
group_groupinvite_df = pd.read_sql(query_groupinvite, conn)
group_groupinvite_df

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,email,status,created_at,updated_at,group_id,user_id,invited_by_id
0,2,None,20,2019-09-10 15:59:33.490257+00:00,2019-09-10 16:36:16.351336+00:00,175,6218,2
1,3,None,20,2019-09-10 16:00:31.655272+00:00,2019-09-10 18:48:19.756997+00:00,175,8328,2
2,1,None,20,2019-09-10 15:41:35.198606+00:00,2019-09-11 07:21:00.675526+00:00,175,6294,2
3,6,None,10,2019-09-11 08:12:56.115308+00:00,2019-09-11 08:12:56.115330+00:00,31,5411,2
4,8,None,20,2019-09-11 21:05:40.445499+00:00,2019-09-11 22:24:21.599480+00:00,175,510,2
...,...,...,...,...,...,...,...,...
393,420,None,20,2023-04-19 18:52:46.022545+00:00,2023-04-19 19:57:54.151530+00:00,932,13467,2
394,422,None,20,2023-05-25 06:06:40.623903+00:00,2023-05-25 22:21:11.015668+00:00,1309,13523,2
395,421,None,20,2023-05-25 06:06:13.441424+00:00,2023-05-25 23:32:12.239884+00:00,1309,12870,2
396,423,None,10,2023-06-02 20:20:32.576387+00:00,2023-06-02 20:20:32.576411+00:00,1309,13407,2


In [14]:
group_groupinvite_df.email.unique()

array([None], dtype=object)

No need to anonymize the `group_groupinvite` table currently since the `email` column consists of no emails.

### matching_matchingsettings

In [15]:
query_matching ='select * from matching_matchingsettings'
matching_matchingsettings_df = pd.read_sql(query_matching, conn)
matching_matchingsettings_df

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,percentage,email
0,1,True,True


No anonymization needed.

### member_user

In [17]:
query_memberuser = "select * from member_user" 
member_user_df = pd.read_sql(query_memberuser, conn)

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [18]:
# column names
member_user_df.columns

Index(['id', 'password', 'last_login', 'is_superuser', 'first_name',
       'last_name', 'is_staff', 'is_active', 'date_joined', 'email',
       'middle_name', 'avatar', 'date_of_birth', 'phone_number',
       'avatar_body_color', 'avatar_hair_color', 'timezone_name',
       'timezone_difference', 'confirmation_hash', 'deleted_at', 'updated_at',
       'avatar_body_id', 'avatar_eyes_id', 'avatar_hair_id', 'avatar_mouth_id',
       'unique_hash', 'password_reset_hash', 'location_id', 'gender_id',
       'scoring_data', 'old_beta_import', 'password_updated_at', 'on_hold_end',
       'on_hold_start', 'invisible', 'tos_acceptance', 'unlocked_categories',
       'available_categories', 'date_joined_beta', 'payment_customer_id',
       'weight_loss', 'measurement_unit', 'segmentation_id', 'last_online',
       'last_seen_buddy_requests', 'last_seen_group_invites',
       'last_phone_verification_sent', 'phone_number_verified',
       'phone_verification_hash', 'last_seen_challenge_invites',


In [19]:
import hashlib

In [20]:
member_user_anonydf = member_user_df.copy()

member_user_anonydf['first_name'] = member_user_anonydf["id"].apply(lambda x: 'FirstUser{}'.format(x)) #first_name

member_user_anonydf['last_name'] = member_user_anonydf["id"].apply(lambda x: 'LastUser{}'.format(x)) #last_name

member_user_anonydf['email'] = member_user_anonydf["id"].apply(lambda x: 'user{}example@.com'.format(x)) #email

if 'phone_number' in member_user_anonydf.columns:
    member_user_anonydf['phone_number'] = member_user_anonydf['id'].apply(lambda x: '123456{}'.format(x)) #phone
    
if 'password' in member_user_anonydf.columns:
    member_user_anonydf['password'] = hashlib.sha256(password.encode('utf-8')).hexdigest() #password


In [21]:
member_user_anonydf.to_sql('member_user_anonymized', engine, if_exists='replace', index=False)

413

### social_auth_code

In [22]:
query_social = "select * from social_auth_code" 
social_auth_df = pd.read_sql(query_social, conn)
social_auth_df

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,id,email,code,verified,timestamp


### member_location

In [24]:
query_location = "select * from member_location" 
member_location_df = pd.read_sql(query_location, conn)

/Users/ruddysimonpour/opt/anaconda3/envs/pythonenv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [25]:
import random

def anonymize_street_address(formatted_address: str) -> str:

    parts = formatted_address.split(",")
    if len(parts) < 4:
        return "123 Anonymized St, Anonymized City, Anonymized State, 00000"

    random_street_name = f"{random.randint(1,999)} Anonymized St"

    generalized_city = "Anonymized City"

    masked_state_code = "Anonymized State"

    anonymized_zip_code = "00000"

    return f"{random_street_name}, {generalized_city}, {masked_state_code}, {anonymized_zip_code}"



In [26]:
member_location_anonydf = member_location_df.copy()

member_location_anonydf['formatted_name'] = member_location_anonydf['formatted_name'].\
apply(anonymize_street_address)


In [27]:
member_location_anonydf.to_sql('member_location_anonymized', engine, if_exists='replace', index=False)

337